In [21]:
import psycopg2
import pandas as pd
import plotly.graph_objects as go
from flask import Flask, jsonify

In [23]:
# Set up database connection parameters
dbname = "E-Commerce_Transactions_db"
user = "postgres"
password = "postgres"
host = "localhost"  # or the IP address of your database server
port = "5432"       # default PostgreSQL port

# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Database connection established.")

     # Create a cursor object using the connection
    cursor = connection.cursor()
    
    # Execute a query
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print("PostgreSQL version:", db_version)
    
  
except Exception as e:
    print("An error occurred:", e)

Database connection established.
PostgreSQL version: ('PostgreSQL 15.6, compiled by Visual C++ build 1937, 64-bit',)


In [24]:
# Select all rows in the table
cursor.execute("SELECT * FROM ecommerce_transactions;")

In [25]:
# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names
column_names = [desc[0] for desc in cursor.description]

In [26]:
# Set the column width so all information in cells are visible
pd.set_option('display.max_colwidth', 100)

## Data Cleaning 

In [27]:
# Create dataframe
ecomm_transactions_df = pd.DataFrame(rows, columns=column_names)

ecomm_transactions_df.head()

,transaction_id,customer_id,transaction_amount,transaction_date,payment_method,product_category,quantity,customer_age,customer_location,device_used,IP_address,shipping_address,billing_address,is_fraudulent,account_age_days,transaction_hour
0,c12e07a0-8a06-4c0d-b5cc-04f3af688570,8ca9f102-02a4-4207-ab63-484e83a1bdf0,42.32,2024-03-24,PayPal,electronics,1,40,East Jameshaven,desktop,110.87.246.85,"5399 Rachel Stravenue Suite 718\nNorth Blakeburgh, IL 78600","5399 Rachel Stravenue Suite 718\nNorth Blakeburgh, IL 78600",0,282,23
1,7d187603-7961-4fce-9827-9698e2b6a201,4d158416-caae-4b09-bd5b-15235deb9129,301.34,2024-01-22,credit card,electronics,3,35,Kingstad,tablet,14.73.104.153,"5230 Stephanie Forge\nCollinsbury, PR 81853","5230 Stephanie Forge\nCollinsbury, PR 81853",0,223,0
2,f2c14f9d-92df-4aaf-8931-ceaf4e63ed72,ccae47b8-75c7-4f5a-aa9e-957deced2137,340.32,2024-01-22,debit card,toys & games,5,29,North Ryan,desktop,67.58.94.93,"195 Cole Oval\nPort Larry, IA 58422","4772 David Stravenue Apt. 447\nVelasquezside, KS 67650",0,360,8
3,e9949bfa-194d-486b-84da-9565fca9e5ce,b04960c0-aeee-4907-b1cd-4819016adcef,95.77,2024-01-16,credit card,electronics,5,45,Kaylaville,mobile,202.122.126.216,"7609 Cynthia Square\nWest Brenda, NV 23016","7609 Cynthia Square\nWest Brenda, NV 23016",0,325,20
4,7362837c-7538-434e-8731-0df713f5f26d,de9d6351-b3a7-4bc7-9a55-8f013eb66928,77.45,2024-01-16,credit card,clothing,5,42,North Edwardborough,desktop,96.77.232.76,"2494 Robert Ramp Suite 313\nRobinsonport, AS 52039","2494 Robert Ramp Suite 313\nRobinsonport, AS 52039",0,116,15


In [28]:
ecomm_transactions_df['shipping_address'] = ecomm_transactions_df['shipping_address'].str.replace('\n', ', ')
ecomm_transactions_df['billing_address'] = ecomm_transactions_df['billing_address'].str.replace('\n', ', ')
ecomm_transactions_df.head()


,transaction_id,customer_id,transaction_amount,transaction_date,payment_method,product_category,quantity,customer_age,customer_location,device_used,IP_address,shipping_address,billing_address,is_fraudulent,account_age_days,transaction_hour
0,c12e07a0-8a06-4c0d-b5cc-04f3af688570,8ca9f102-02a4-4207-ab63-484e83a1bdf0,42.32,2024-03-24,PayPal,electronics,1,40,East Jameshaven,desktop,110.87.246.85,"5399 Rachel Stravenue Suite 718, North Blakeburgh, IL 78600","5399 Rachel Stravenue Suite 718, North Blakeburgh, IL 78600",0,282,23
1,7d187603-7961-4fce-9827-9698e2b6a201,4d158416-caae-4b09-bd5b-15235deb9129,301.34,2024-01-22,credit card,electronics,3,35,Kingstad,tablet,14.73.104.153,"5230 Stephanie Forge, Collinsbury, PR 81853","5230 Stephanie Forge, Collinsbury, PR 81853",0,223,0
2,f2c14f9d-92df-4aaf-8931-ceaf4e63ed72,ccae47b8-75c7-4f5a-aa9e-957deced2137,340.32,2024-01-22,debit card,toys & games,5,29,North Ryan,desktop,67.58.94.93,"195 Cole Oval, Port Larry, IA 58422","4772 David Stravenue Apt. 447, Velasquezside, KS 67650",0,360,8
3,e9949bfa-194d-486b-84da-9565fca9e5ce,b04960c0-aeee-4907-b1cd-4819016adcef,95.77,2024-01-16,credit card,electronics,5,45,Kaylaville,mobile,202.122.126.216,"7609 Cynthia Square, West Brenda, NV 23016","7609 Cynthia Square, West Brenda, NV 23016",0,325,20
4,7362837c-7538-434e-8731-0df713f5f26d,de9d6351-b3a7-4bc7-9a55-8f013eb66928,77.45,2024-01-16,credit card,clothing,5,42,North Edwardborough,desktop,96.77.232.76,"2494 Robert Ramp Suite 313, Robinsonport, AS 52039","2494 Robert Ramp Suite 313, Robinsonport, AS 52039",0,116,15


In [29]:
clean_transactn_df = ecomm_transactions_df.copy()

clean_transactn_df['shipping_state'] = ecomm_transactions_df['shipping_address'].str.extract(r',\s*([A-Za-z]{2})\s*\d{5}')
clean_transactn_df['billing_state'] = ecomm_transactions_df['billing_address'].str.extract(r',\s*([A-Za-z]{2})\s*\d{5}')

clean_transactn_df.head()

,transaction_id,customer_id,transaction_amount,transaction_date,payment_method,product_category,quantity,customer_age,customer_location,device_used,IP_address,shipping_address,billing_address,is_fraudulent,account_age_days,transaction_hour,shipping_state,billing_state
0,c12e07a0-8a06-4c0d-b5cc-04f3af688570,8ca9f102-02a4-4207-ab63-484e83a1bdf0,42.32,2024-03-24,PayPal,electronics,1,40,East Jameshaven,desktop,110.87.246.85,"5399 Rachel Stravenue Suite 718, North Blakeburgh, IL 78600","5399 Rachel Stravenue Suite 718, North Blakeburgh, IL 78600",0,282,23,IL,IL
1,7d187603-7961-4fce-9827-9698e2b6a201,4d158416-caae-4b09-bd5b-15235deb9129,301.34,2024-01-22,credit card,electronics,3,35,Kingstad,tablet,14.73.104.153,"5230 Stephanie Forge, Collinsbury, PR 81853","5230 Stephanie Forge, Collinsbury, PR 81853",0,223,0,PR,PR
2,f2c14f9d-92df-4aaf-8931-ceaf4e63ed72,ccae47b8-75c7-4f5a-aa9e-957deced2137,340.32,2024-01-22,debit card,toys & games,5,29,North Ryan,desktop,67.58.94.93,"195 Cole Oval, Port Larry, IA 58422","4772 David Stravenue Apt. 447, Velasquezside, KS 67650",0,360,8,IA,KS
3,e9949bfa-194d-486b-84da-9565fca9e5ce,b04960c0-aeee-4907-b1cd-4819016adcef,95.77,2024-01-16,credit card,electronics,5,45,Kaylaville,mobile,202.122.126.216,"7609 Cynthia Square, West Brenda, NV 23016","7609 Cynthia Square, West Brenda, NV 23016",0,325,20,NV,NV
4,7362837c-7538-434e-8731-0df713f5f26d,de9d6351-b3a7-4bc7-9a55-8f013eb66928,77.45,2024-01-16,credit card,clothing,5,42,North Edwardborough,desktop,96.77.232.76,"2494 Robert Ramp Suite 313, Robinsonport, AS 52039","2494 Robert Ramp Suite 313, Robinsonport, AS 52039",0,116,15,AS,AS


In [34]:
  # Close the cursor and connection
cursor.close()
connection.close()

## Visualizations

In [14]:
# Extract state information
states = pd.concat([clean_transactn_df['shipping_state'], clean_transactn_df['billing_state']], axis=0)

# Remove duplicates and sort
states = states.drop_duplicates().sort_values()

In [15]:
# Group by state and count fraudulent transactions
fraudulent_transactions_by_state = clean_transactn_df[clean_transactn_df['is_fraudulent'] == 1].groupby('shipping_state').size()

fraudulent_transactions_by_state


shipping_state
AK    14
AL    20
AR    17
AS    17
AZ    19
CA    21
CO    26
CT    15
DC    10
DE    19
FL    20
FM    19
GA    14
GU    21
HI    17
IA    21
ID    24
IL    20
IN    14
KS    24
KY    14
LA    26
MA    11
MD    12
ME    18
MH    24
MI    31
MN    16
MO    16
MP    17
MS    17
MT    26
NC    19
ND    16
NE    25
NH    13
NJ    23
NM     9
NV    23
NY    17
OH    18
OK    14
OR    17
PA    23
PR    12
PW    23
RI    21
SC    14
SD    25
TN    17
TX    15
UT    26
VA    22
VI    21
VT    21
WA    23
WI    21
WV    18
WY    21
dtype: int64

In [16]:
# Create map visualization
state_map = go.Figure(data=go.Choropleth(
    locations=states,
    z=fraudulent_transactions_by_state,
    locationmode='USA-states',
    colorscale='Reds',
    colorbar_title='Fraudulent Transactions'
))

state_map.update_layout(
    title_text='Fraudulent Transactions by State',
    geo_scope='usa',
)

In [ ]:
# Set up database connection parameters
dbname = "E-Commerce_Transactions_db"
user = "postgres"
password = "postgres"
host = "localhost"  # or the IP address of your database server
port = "5432"       # default PostgreSQL port

# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Database connection established.")

     # Create a cursor object using the connection
    cursor = connection.cursor()
    
    # Execute a query
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print("PostgreSQL version:", db_version)
    
  
except Exception as e:
    print("An error occurred:", e)

## Build Web Application